LOADING DATA

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import multiprocessing
import functools

In [ ]:
def read_csv(img,data,lbl,rot,id):
    rows = open('dataset4_csv/ID_'+str(id)+'.csv').read().strip().split('\n')
    for row in rows:
        row = row.split(',')
        (x,y,label,rotation,pathimg) = row
        image =  load_img(pathimg, target_size=(256, 256))
        image = img_to_array(image)
        img.append(image)
        data.append([x,y])
        lbl.append(label)
        rot.append(rotation)

In [ ]:
if __name__ == '__main__':
    
    manager = multiprocessing.Manager()
    img = manager.list()
    data = manager.list()
    lbl = manager.list()
    rot = manager.list()

    id = list(range(1,5)) #change here

    partial_read_csv = functools.partial(read_csv, img, data, lbl, rot)

    pool = multiprocessing.Pool()
    pool.map(partial_read_csv, id)

In [ ]:
pool.close()
pool.terminate()

In [ ]:
img = np.array(img,dtype='float32')
data = np.array(data,dtype='float32')
lbl = np.array(lbl)
rot = np.array(rot)
lb = LabelBinarizer()
lbl = lb.fit_transform(lbl)
rot = lb.fit_transform(rot)

In [ ]:
split = train_test_split(img, lbl, rot, data, test_size=0.1)
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainRotations, testRotations) = split[4:6]
(trainCoordinates, testCoordinates) = split[6:8]
split = train_test_split(trainImages, trainLabels, trainRotations, trainCoordinates, test_size=0.1)
(trainImages, valImages) = split[:2]
(trainLabels, valLabels) = split[2:4]
(trainRotations, valRotations) = split[4:6]
(trainCoordinates, valCoordinates) = split[6:8]

MODEL CREATION

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Rescaling, Dropout, BatchNormalization
from keras.utils.vis_utils import plot_model

In [ ]:
#input layer
inputs = Input(shape=(256,256,3))
scaled_inputs = Rescaling(1/255)(inputs)

#label branch
l = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(scaled_inputs)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.2)(l)
l = Conv2D(32, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(l)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.3)(l)
l = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(l)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.4)(l)
l = Flatten()(l)
l = Dense(128, activation='relu', kernel_initializer='he_uniform')(l)
l = BatchNormalization()(l)
l = Dropout(0.5)(l)
l = Dense(4, activation = 'softmax', name='label')(l) #change here

#rot branch
r = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(scaled_inputs)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.2)(r)
r = Conv2D(32, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(r)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.3)(r)
r = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(r)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.4)(r)
r = Flatten()(r)
r = Dense(128, activation='relu', kernel_initializer='he_uniform')(r)
r = BatchNormalization()(r)
r = Dropout(0.5)(r)
r = Dense(15, activation = 'softmax', name='rotation')(r)

#ann branch
a = Conv2D(16, (3,3), 1, activation = 'relu')(scaled_inputs)
a = MaxPooling2D()(a)
a = Conv2D(32, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Conv2D(64, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Conv2D(128, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Flatten()(a)
a = Dense(32, activation = 'linear')(a)
a = Dense(32, activation = 'linear', kernel_regularizer='l2', bias_regularizer='l2')(a)
a = Dense(32, activation = 'linear', kernel_regularizer='l2', bias_regularizer='l2')(a)
a = Dense(32, activation = 'linear')(a)
a = Dense(2, activation = 'linear', name='coordinate')(a)

#model
model = Model(
	inputs=inputs,
	outputs=(l, r, a),
    compile=False)

#compile
model.compile(loss={
	'label': 'categorical_crossentropy',
	'rotation': 'categorical_crossentropy',
	'coordinate': 'mse',
}, optimizer='adam', metrics=['accuracy'])

plot_model(model)

TRAINING

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
hist = model.fit(
	trainImages, {
	'label': trainLabels,
	'rotation': trainRotations,
	'coordinate': trainCoordinates,
},
	validation_data= (valImages, {
	'label': valLabels,
	'rotation': valRotations,
	'coordinate': valCoordinates,
}),
	batch_size=32,
	epochs=20)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.show()

TRAINING FROM H5

In [ ]:
import numpy as np
import cv2
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.utils.vis_utils import plot_model

In [ ]:
model = load_model('super1.h5', compile=False)

model.compile(loss={
	'label': 'categorical_crossentropy',
	'rotation': 'categorical_crossentropy',
	'coordinate': 'mse',
}, optimizer='adam', metrics=['accuracy'])
plot_model(model)

In [ ]:
hist = model.fit(
	trainImages, {
	'label': trainLabels,
	'rotation': trainRotations,
	'coordinate': trainCoordinates,
},
	validation_data= (valImages, {
	'label': valLabels,
	'rotation': valRotations,
	'coordinate': valCoordinates,
}),
	batch_size=32,
	epochs=20)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('loss3.PNG')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['label_accuracy'], color='teal', label='label_accuracy')
plt.plot(hist.history['val_label_accuracy'], color='orange', label='val_label_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('label3acc.PNG')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['rotation_accuracy'], color='teal', label='rotation_accuracy')
plt.plot(hist.history['val_rotation_accuracy'], color='orange', label='val_rotation_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('rotation3acc.PNG')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['coordinate_accuracy'], color='teal', label='coordinate_accuracy')
plt.plot(hist.history['val_coordinate_accuracy'], color='orange', label='val_coorrdinate_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('coordinate3acc.PNG')

TESTING

In [ ]:
import numpy as np
import cv2
from keras.models import load_model
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, r2_score

In [ ]:
model = load_model('super1.h5', compile=False)

model.compile(loss={
	'label': 'categorical_crossentropy',
	'rotation': 'categorical_crossentropy',
	'coordinate': 'mse',
}, optimizer='adam', metrics=['accuracy'])

yhat = model.predict(testImages)
y_true = [testLabels,testRotations,testCoordinates]

In [ ]:
#label,rotation,(x,y)
accuracy_score(np.amax(y_true[0],axis=1),np.amax(yhat[0].round(),axis=1)),accuracy_score(np.amax(y_true[1],axis=1),np.amax(yhat[1].round(),axis=1)),r2_score(y_true[2],yhat[2])

SINGLE TESTING

In [ ]:
image = cv2.imread('test_images/96trans50_-150.jpg')
image_show = np.array(image, dtype='float32')
height, width = image_show.shape[:2]
image_input = cv2.resize(image_show, (256,256))
plt.imshow(image_show)

In [ ]:
y = model.predict(np.expand_dims(image_input,0))
labels = ['ID_1', 'ID_2', 'ID_3']
rotations = [0,120,144,168,192,216,24,240,264,288,312,336,48,72,96]
label = y[0].argmax()
rotation = y[1].argmax()
co_x = y[2][0][0]
co_y = y[2][0][1]
cv2.circle(image_show, (int(np.around(co_x*width+width/2)),int(np.around(co_y*height+height/2))), 190, (255,255,255), thickness=2)
cv2.putText(image_show, str(labels[label]) + ' ' + str(rotations[rotation]) + ' degree', (100,100), cv2.FONT_HERSHEY_TRIPLEX, 4, (255,255,255), thickness=2)
plt.imshow(image_show)

CONVERTING TO TFLITE

In [ ]:
TF_LITE_MODEL_FILE_NAME = "model.tflite"
tf_lite_converter = tensorflow.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tensorflow.lite.Optimize.OPTIMIZE_FOR_SIZE]
tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)